In [2]:
import re

def extract_title_and_link(content):
    # Regular expression to extract the title and link
    pattern = r'\[\s*(.*?)\s*\]\(\s*(.*?)\s*\)'

    # Search for the pattern in the content
    match = re.search(pattern, content)

    if match:
        title = match.group(1)
        link = match.group(2)
        return title, link
    else:
        return None, None

# Sample line from the markdown content
content = "1364. [2024-[Strategic] - DMS](https://aws.highspot.com/items/660029a5d545f02ded7055a7?lfrm=shp.684)"

# Extract title and link
title, link = extract_title_and_link(content)
print(f"Title: {title}")
print(f"Link: {link}")

Title: 2024-[Strategic] - DMS
Link: https://aws.highspot.com/items/660029a5d545f02ded7055a7?lfrm=shp.684


In [3]:
import re

def extract_titles(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # Use regex to find all titles starting with ###
    titles = re.findall(r'### (.+)', content)
    return titles

file_path = '/Users/ianleely/Documents/Codes/aws-genai-try-it-on/output/title.md'
titles = extract_titles(file_path)
print(len(titles), titles[:5])

1757 ['Medical Insights Hub[v1.5.1] FCD - 中文', '金融 - GCR Financial Services Homepage', '下一代智能搜索和知识库解决方案指南 | 智能知识库-制造业解决方案', 'GPU - GCR GenAI Hub', '零售和快消 - RCH Solution List']


In [8]:
import json

def process_file(file_path, title_list):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    documents = []
    current_doc = None

    for line in lines:
        if not line.strip():
            continue

        # parts = line.split(']')
        # name = parts[0].split('[')[1]
        # url = parts[1].split('(')[1].split(')')[0]
        name, url = extract_title_and_link(line)

        if name in title_list:
            if current_doc:
                documents.append(current_doc)
            current_doc = {
                "title": name,
                "link": url,
                "types": '',
                "materials": []
            }
        elif current_doc and url == current_doc["link"]:
            current_doc["types"] = name
        elif current_doc:
            current_doc["materials"].append({"name": name, "url": url})

    if current_doc:
        documents.append(current_doc)

    return documents

In [11]:
import json

# Example usage
title_list = titles
file_path = '/Users/ianleely/Documents/Codes/aws-genai-try-it-on/output/20250305_180730_highspot_interactive.md'
result = process_file(file_path, title_list)
# Save result to a jsonl file
output_file_path = '/Users/ianleely/Documents/Codes/aws-genai-try-it-on/output/result.jsonl'
with open(output_file_path, 'w', encoding='utf-8') as f:
    for entry in result:
        f.write(json.dumps(entry, ensure_ascii=False) + '\n')